In [1]:
!pip install langchain langchain_experimental openai duckduckgo-search youtube_search wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 15.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 56.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=6cfe23072d7999e714df4c9abbde5130b335f3bf11a77327727f5

In [18]:
# Import the required modules
import os
import getpass
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [19]:
os.environ["OPENAI_API_KEY"] = os.environ.get("gpt_4_key")

* Define the tools the agent will use to execute its sub-tasks
* These tools can be functions or APIs that perform specific actions
* For example, you can define a tool for searching the internet or performing calculations

In [20]:
# Create a language model object using ChatOpenAI
llm = ChatOpenAI(model="gpt-4-1106-preview",temperature=0) 

# Create a math chain object using the language model
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True) 

In [21]:
# Create a search object using DuckDuckGoSearchRun
search = DuckDuckGoSearchRun() 

# Create a tool object for searching
search_tool = Tool( 
    name="Search", # Give the tool a name
    func=search.run, # Assign the function to run the tool
    description="Useful for answering questions about up to the minute and current events" # Provide a description of the tool
)

In [22]:
# Create a tool object for calculating
calcuator_tool = Tool( 
    name="Calculator", # Give the tool a name
    func=llm_math_chain.run, # Assign the function to run the tool
    description="Useful for when you need to do math" # Provide a description of the tool
)
agent_tools = [search_tool, calcuator_tool] # Create a list of the tools

* Define the plan method for the agent
*  This method takes in the intermediate steps taken by the agent and user inputs as arguments
*  It should analyze the current state and decide what action or tool to use next
*  The plan method should return a list of AgentAction objects specifying the tools to use

In [23]:
# Create a language model object using ChatOpenAI
model = ChatOpenAI(model="gpt-4",temperature=0)

In [24]:
# Load the chat planner object using the language model
planner = load_chat_planner(model)
# Access the content of the messages in the prompt
planner.llm_chain.prompt.messages[0].content 

"Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'"

* Create an instance of the agent executor class, passing in the agent and tools as arguments
* The agent executor handles the execution of the agent’s actions and tools

In [25]:
# Load the agent executor object using the language model and the tools
executor = load_agent_executor(model, agent_tools, verbose=True)
# Create a plan-and-execute agent object using the planner and the executor
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

# Print the template of the prompt messages
print(executor.chain.agent.llm_chain.prompt.messages[0].prompt.template) 

# Run the agent with the user query

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: Useful for answering questions about up to the minute and current events, args: {{'tool_input': {{'type': 'string'}}}}
Calculator: Useful for when you need to do math, args: {{'tool_input': {{'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```

Begin! Reminder to ALWAYS respond with a valid json bl

* Run the agent with a user query
* The agent will respond to the human as helpfully and accurately as possible
* The agent will use the tools if necessary or respond directly if appropriate
* The agent will follow the format of Question, Thought, Action, Observation, and Final Answer

### Example 1

In [26]:
agent.run("Who is the current Premier of Manitoba? What is their current age raised to the 0.43 power?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the current Premier of Manitoba.'), Step(value='Find out their current age.'), Step(value='Calculate the age raised to the 0.43 power.'), Step(value='Given the above steps taken, please respond to the users original question.\n')]

> Entering new AgentExecutor chain...
Thought: I need to find out who the current Premier of Manitoba is. I can use the search tool to find this information.
Action:
```
{
  "action": "Search",
  "action_input": "current Premier of Manitoba"
}
```
Observation: Heather Dorothy Stefanson MLA (born May 11, 1970) is a Canadian politician who served as the 24th premier of Manitoba from 2021 to 2023; the first woman in the province's history to hold that role.. She is the outgoing leader of the Progressive Conservative Party of Manitoba and serves as Leader of the Opposition of Manitoba. She sits as a member of the Legislative Assembly (MLA) for the ... Manitoba had a Metis premier in the late 18

'The age of the current Premier of Manitoba, when raised to the power of 0.43, is approximately 4.99.'

### Example 2

In [28]:
agent.run("What is cristiano ronaldo's age divided by messi's age")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the current ages of Cristiano Ronaldo and Lionel Messi.'), Step(value="Divide Cristiano Ronaldo's age by Lionel Messi's age."), Step(value='Round the result to two decimal places for precision.'), Step(value='Given the above steps taken, please respond to the users original question.\n')]

> Entering new AgentExecutor chain...
Thought: I need to find the current ages of Cristiano Ronaldo and Lionel Messi.
Action:
```
{
  "action": "Search",
  "action_input": "current age of Cristiano Ronaldo"
}
```
Observation: Ronaldo, a five-time FIFA player of the year, in 2020 became the first active team-sport athlete to surpass $1 billion in career earnings. In addition to a lifetime deal with Nike, Ronaldo makes ... Cristiano Ronaldo age. As of February 27, 2024, Cristiano Ronaldo is 39 years old. He was born on February 5, 1985, in Funchal, Madeira, Portugal. Cristiano Ronaldo height. Cristiano Ronaldo's height is approximatel

"As of 2024, Cristiano Ronaldo is 39 years old and Lionel Messi is 36 years old. When we divide Cristiano Ronaldo's age by Lionel Messi's age, the result is approximately 1.08."